In [1]:
import matplotlib.pyplot as plt
import numpy as np
%matplotlib tk

In [29]:
Nm = 2
Np = 80

x = np.arange(Nm)
x[x>Nm/2] = x[x>Nm/2]-Nm
X, Y, Z = np.meshgrid(x, x, x, sparse=False, indexing='ij')

r = np.sqrt(X**2+Y**2+Z**2)

point_phi = 1. / (r+2.) * 6.67408 * 10e-11  # m3 kg-1 c-2
point_phi_fft = np.fft.rfftn(point_phi)

plt.imshow(np.log10(abs(point_phi_fft[:,:,0])), origin = 'lower')

In [33]:
def c2i(cellx, celly, cellz, Nm, shift=[0,0,0] ) :
    '''Return the indices of the neighbor grid cell in the shift
    direction of each particle.'''

    # Shift to the coordinates of a neighbor cell
    shiftx = cellx+shift[0]
    shifty = celly+shift[1]
    shiftz = cellz+shift[2]
    
    # Impose BCs
    shiftx[shiftx>=Nm] -= Nm
    shifty[shifty>=Nm] -= Nm
    shiftz[shiftz>=Nm] -= Nm
    
    shiftx[shiftx<0] += Nm
    shifty[shifty<0] += Nm
    shiftz[shiftz<0] += Nm
    
    return (shiftx*Nm*Nm + shifty*Nm + shiftz)

def calc_yweights(xp, yp, zp) :
    '''This function calculates yijk, which is a product of the
    d's and t's'''

    cellx = np.floor(xp).astype(int)
    celly = np.floor(yp).astype(int)
    cellz = np.floor(zp).astype(int)
    
    dx = xp - cellx
    dy = yp - celly
    dz = zp - cellz
    
    tx, ty, tz = 1-dx, 1-dy, 1-dz
    
    # Calculate volume overlap for weights
    y000=tx*ty*tz
    y010=tx*dy*tz
    
    y001=tx*ty*dz
    y011=tx*dy*dz
    
    y110=dx*dy*tz
    y100=dx*ty*tz
    
    y101=dx*ty*dz
    y111=dx*dy*dz
    
    return cellx, celly, cellz, y000, y010, y001, y011, y110, y100, y101, y111
    
def DensityEstimation( xp, yp, zp, Nm, Np) :
    ''' Use CIC weighting to estimate the density at each
    particle's position '''

    cellx, celly, cellz, y000, y010, y001, y011, y110, y100, y101, y111 = calc_yweights(xp, yp, zp)

  
    c2i000=c2i(cellx, celly, cellz, Nm, shift=[0,0,0])
    c2i001=c2i(cellx, celly, cellz, Nm, shift=[0,0,1])
    c2i100=c2i(cellx, celly, cellz, Nm, shift=[1,0,0])
    c2i110=c2i(cellx, celly, cellz, Nm, shift=[1,1,0])
    
    
    c2i010=c2i(cellx, celly, cellz, Nm, shift=[0,1,0])
    c2i101=c2i(cellx, celly, cellz, Nm, shift=[1,0,1])
    c2i111=c2i(cellx, celly, cellz, Nm, shift=[1,1,1])
    c2i011=c2i(cellx, celly, cellz, Nm, shift=[0,1,1])
    
    
    mdens = np.zeros([Nm**3])
    mdens+=np.bincount(c2i000, weights=y000, minlength=Nm**3)
    mdens+=np.bincount(c2i001, weights=y001, minlength=Nm**3)
    mdens+=np.bincount(c2i100, weights=y100, minlength=Nm**3)
    mdens+=np.bincount(c2i110, weights=y110, minlength=Nm**3)
      
        
    mdens+=np.bincount(c2i010, weights=y010, minlength=Nm**3)
    mdens+=np.bincount(c2i101, weights=y101, minlength=Nm**3)
    mdens+=np.bincount(c2i111, weights=y111, minlength=Nm**3)
    mdens+=np.bincount(c2i011, weights=y011, minlength=Nm**3)

    mdens=mdens*Nm**3/Np**3
    return mdens.reshape([Nm, Nm, Nm])


def GetPhi(mdens, point_phi_fft):
    mdens_fft = np.fft.rfftn(mdens)
    mdens_fft *= point_phi_fft
    phi = np.fft.irfftn(mdens_fft)
    return phi



def GetFg(xp,yp,zp, phi,Nm):
    gx = np.gradient(phi,axis=0)
    gy = np.gradient(phi,axis=1)
    gz = np.gradient(phi,axis=2)
    
    cellx, celly, cellz, y000, y010, y001, y011, y110, y100, y101, y111 = calc_yweights(xp, yp, zp)
    c2i000=c2i(cellx, celly, cellz, Nm, shift=[0,0,0])
    c2i001=c2i(cellx, celly, cellz, Nm, shift=[0,0,1])
    c2i100=c2i(cellx, celly, cellz, Nm, shift=[1,0,0])
    c2i110=c2i(cellx, celly, cellz, Nm, shift=[1,1,0])
    
    
    c2i010=c2i(cellx, celly, cellz, Nm, shift=[0,1,0])
    c2i101=c2i(cellx, celly, cellz, Nm, shift=[1,0,1])
    c2i111=c2i(cellx, celly, cellz, Nm, shift=[1,1,1])
    c2i011=c2i(cellx, celly, cellz, Nm, shift=[0,0,1])
    
    
    fx = gx.flatten()[c2i000]*y000
    fy = gy.flatten()[c2i000]*y000
    fz = gz.flatten()[c2i000]*y000
    
    fx += gx.flatten()[c2i001]*y001
    fy += gy.flatten()[c2i001]*y001
    fz = gz.flatten()[c2i001]*y001
    
    fx += gx.flatten()[c2i010]*y010
    fy += gy.flatten()[c2i010]*y010
    fz = gz.flatten()[c2i010]*y010
    
    fx += gx.flatten()[c2i100]*y100
    fy += gy.flatten()[c2i100]*y100
    fz = gz.flatten()[c2i100]*y100
    
    fx += gx.flatten()[c2i101]*y101
    fy += gy.flatten()[c2i101]*y101
    fz += gy.flatten()[c2i101]*y101
    
    fx += gx.flatten()[c2i111]*y111
    fy += gy.flatten()[c2i111]*y111
    fz += gy.flatten()[c2i111]*y111
    
    fx += gx.flatten()[c2i011]*y011
    fy += gy.flatten()[c2i011]*y011
    fz += gy.flatten()[c2i011]*y011
    
    fx += gx.flatten()[c2i110]*y110
    fy += gy.flatten()[c2i110]*y110
    fz += gy.flatten()[c2i110]*y110
    
    return fx, fy, fz

def step(xp, yp, zp, vxp, vyp, vzp, dt, Nm, Np):
    xp[xp>=Nm] -= Nm
    yp[yp>=Nm] -= Nm
    zp[zp>=Nm] -= Nm
    
    xp[xp<0] += Nm
    yp[yp<0] += Nm
    zp[yp<0] += Nm
    
    mdens = DensityEstimation(xp, yp, zp, Nm, Np)
    phi = GetPhi(mdens, point_phi_fft)
    fx, fy, fz = GetFg(xp, yp, zp, phi, Nm)
    vxp += fx*dt*100
    vyp += fy*dt*100
    vzp += fz*dt*100
    
    xp += vxp*dt
    yp += vyp*dt
    zp += vzp*dt
    return xp, yp, zp, vxp, vyp, vzp

In [37]:
N = Nm

In [6]:
xp = np.array([0.3+N/2, N/2])  # одна в центре
yp = np.array([N/2, N/4])     # ещё одна где-то
zp = np.array([N/2, N/2])

vxp = np.array([0., 1.])
vyp = np.array([0., 0.])
vzp = np.array([0., 0.])
print(xp)
print(yp)

[ 60.3  60. ]
[ 60.  30.]


In [21]:
mdens = DensityEstimation(xp, yp, zp, N, 2)      # две точки на поле запихиивает
plt.imshow(mdens[:,:,60], cmap='gray_r')
print(np.shape(mdens))


(120, 120, 120)


In [28]:
phi = GetPhi(mdens, point_phi_fft)
plt.imshow(np.log10(phi[:,:,68]))



In [38]:
from mpl_toolkits.mplot3d import Axes3D

In [39]:
dt=20
xp = np.array([0.3+N/2, N/2])  
yp = np.array([4+N/2, N/4])     
zp = np.array([6+N/2, N/4])

vxp = np.array([2., 1.1])
vyp = np.array([2., 1.])
vzp = np.array([2.1, 1.])

#vyp = np.random.normal(loc=Nm/2, scale=Nm/1e4, size=[2])
#vzp = np.random.normal(loc=Nm/2, scale=Nm/1e4, size=[2])
#vxp = np.random.normal(loc=Nm/2, scale=Nm/1e4, size=[2])

#import matplotlib
#cmap = matplotlib.cm.get_cmap('jet')

mdens = DensityEstimation(xp, yp, zp, N, 2)
#p = plt.imshow(mdens[:,:,68])
#fig = plt.gcf()

Nstep=60
j=0
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')


for i in range(Nstep):
    
    #rgb = cmap((i+0.5)/Nstep)
    xp, yp, zp, vxp, vyp, vzp = step(xp, yp, zp, vxp, vyp, vzp, dt, N,2)
    mdens = DensityEstimation(xp, yp, zp, N, 2)
    #p.set_data(mdens[:,:,61])
    #plt.pause(0.01)
    #print(xp, yp, zp)
    print(vxp,vyp,vzp)
    ax.clear()
    ax.scatter(xp, yp, zp, marker = 'o')
    ax.set_xlim(0,80)
    ax.set_ylim(0,80)
    ax.set_zlim(0,80)
    ax.set_title('3d')
    plt.savefig("lip_"+str(j)+"_r.jpg", format = "jpg")
    j+=1


ValueError: operands could not be broadcast together with shapes (8,) (16,) (8,) 